<a href="https://colab.research.google.com/github/Hemanthkanakanti/Development-of-Interactive-Cyber-Threat-Visualization-Dashboard/blob/main/SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("--- SQL JOINs Examples ---")

# Ensure Products and Sales tables are set up correctly for joining
# Recreate if necessary for consistent examples

# First, close existing connection if open from previous examples
# if 'conn' in globals() and conn:
#     conn.close()
# conn = sqlite3.connect(':memory:')
# cursor = conn.cursor()

# Re-create Products and Sales tables and populate data (ensure product_id is primary key and sales.product_id is foreign key)
cursor.execute('DROP TABLE IF EXISTS Sales;')
cursor.execute('DROP TABLE IF EXISTS Products;')

cursor.execute('''
CREATE TABLE Products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    category TEXT,
    price REAL,
    stock_quantity INTEGER
);
''')
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Laptop', 'Electronics', 1200.00, 50);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Mouse', 'Electronics', 25.50, 200);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Keyboard', 'Electronics', 75.00, 150);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Desk Chair', 'Furniture', 150.00, 30);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Monitor', 'Electronics', 300.00, 80);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Coffee Table', 'Furniture', 80.00, 40);")
conn.commit()

cursor.execute('''
CREATE TABLE Sales (
    sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_id INTEGER,
    sale_date DATE,
    quantity_sold INTEGER,
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);
''')
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (1, '2023-10-01', 5);") # Laptop
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (2, '2023-10-02', 10);") # Mouse
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (1, '2023-10-03', 3);") # Laptop
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (4, '2023-10-03', 2);") # Desk Chair
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (7, '2023-10-05', 1);") # Non-existent product_id to demonstrate LEFT JOIN
conn.commit()

print("Tables 'Products' and 'Sales' refreshed for JOIN examples.")

# 1. INNER JOIN: Get all sales transactions with product details
print("\n--- INNER JOIN: Sales with matching product details ---")
query_inner_join = '''
SELECT
    s.sale_id,
    p.product_name,
    p.category,
    s.quantity_sold,
    (p.price * s.quantity_sold) AS total_sale_amount
FROM
    Sales s
INNER JOIN
    Products p ON s.product_id = p.product_id
;'''
cursor.execute(query_inner_join)
df_inner_join = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_inner_join)

# 2. LEFT JOIN: Get all products and their sales (if any)
print("\n--- LEFT JOIN: All products and their sales (NULL if no sale) ---")
query_left_join = '''
SELECT
    p.product_name,
    p.category,
    s.sale_id,
    s.quantity_sold
FROM
    Products p
LEFT JOIN
    Sales s ON p.product_id = s.product_id
ORDER BY p.product_name, s.sale_id
;'''
cursor.execute(query_left_join)
df_left_join = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_left_join)

In [ ]:
print("--- SQL CASE Statement Example ---")

# Use a CASE statement to categorize products by price
query_case = '''
SELECT
    product_name,
    price,
    CASE
        WHEN price >= 500 THEN 'High-End'
        WHEN price >= 100 AND price < 500 THEN 'Mid-Range'
        ELSE 'Budget'
    END AS price_category
FROM
    Products
ORDER BY price DESC;
'''
cursor.execute(query_case)
df_price_category = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_price_category)

# Another example: Update product name based on category (conceptual, would use UPDATE statement)
# SELECT product_name, category,
#        CASE category
#            WHEN 'Electronics' THEN product_name || ' (Electronic)'
#            WHEN 'Furniture' THEN product_name || ' (Home Goods)'
#            ELSE product_name
#        END AS display_name
# FROM Products;


In [ ]:
print("--- SQL Subqueries Examples ---")

# 1. Subquery in the WHERE clause: Find products that have been sold.
print("\n--- Subquery in WHERE clause: Products that have been sold ---")
query_sub_where = '''
SELECT
    product_name,
    category,
    price
FROM
    Products
WHERE
    product_id IN (SELECT DISTINCT product_id FROM Sales);
'''
cursor.execute(query_sub_where)
df_sold_products = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_sold_products)

# 2. Subquery in the FROM clause (Derived Table): Find categories with average price above a certain value.
print("\n--- Subquery in FROM clause (Derived Table): Categories with average price > 100 ---")
query_sub_from = '''
SELECT
    category,
    avg_price
FROM
    (SELECT category, AVG(price) AS avg_price FROM Products GROUP BY category)
WHERE
    avg_price > 100;
'''
cursor.execute(query_sub_from)
df_avg_price_category = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_avg_price_category)

# 3. Subquery in the SELECT clause (Scalar Subquery): Get product name and its total sales quantity (if any)
print("\n--- Subquery in SELECT clause (Scalar Subquery): Product total sales quantity ---")
query_sub_select = '''
SELECT
    p.product_name,
    p.price,
    (SELECT SUM(s.quantity_sold) FROM Sales s WHERE s.product_id = p.product_id) AS total_quantity_sold
FROM
    Products p
ORDER BY p.product_name;
'''
cursor.execute(query_sub_select)
df_product_sales_scalar = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_product_sales_scalar)

# Close the connection after all examples
conn.close()
print("\nDatabase connection closed.")

## Understanding SQL Commands

SQL (Structured Query Language) is the standard language for managing and manipulating relational databases. It's broadly categorized into four main types of commands:

1.  **DDL (Data Definition Language)**: Used for defining and managing database structures.
2.  **DML (Data Manipulation Language)**: Used for managing data within the database.
3.  **DCL (Data Control Language)**: Used for controlling access and permissions to the database.
4.  **TCL (Transaction Control Language)**: Used for managing transactions within the database.

First, let's set up a simple SQLite database for our examples.

In [ ]:
import sqlite3
import pandas as pd

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

print("Database connection established.")

### 1. DDL (Data Definition Language)

DDL commands are used to define, modify, and delete database objects like tables, indexes, and views. They deal with the database schema rather than the data itself.

*   **`CREATE`**: Used to create new database objects (e.g., `CREATE TABLE`, `CREATE DATABASE`, `CREATE INDEX`).
*   **`ALTER`**: Used to modify the structure of existing database objects (e.g., `ALTER TABLE` to add/drop columns, change data types).
*   **`DROP`**: Used to delete existing database objects (e.g., `DROP TABLE`, `DROP INDEX`).
*   **`TRUNCATE`**: Used to remove all records from a table, but keeps the table structure. It's faster than `DELETE` for removing all rows because it doesn't log individual row deletions.
*   **`RENAME`**: Used to rename an existing database object.

In [ ]:
print("--- DDL Commands Examples ---")

# CREATE TABLE
cursor.execute('''
CREATE TABLE Employees (
    id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    department TEXT,
    hire_date DATE
);
''')
print("Table 'Employees' created.")

# ALTER TABLE - Add a new column
cursor.execute('ALTER TABLE Employees ADD COLUMN salary REAL;')
print("Column 'salary' added to 'Employees' table.")

# RENAME TABLE
cursor.execute('ALTER TABLE Employees RENAME TO Staff;')
print("Table 'Employees' renamed to 'Staff'.")

# CREATE INDEX (for better query performance)
cursor.execute('CREATE INDEX idx_staff_department ON Staff (department);')
print("Index 'idx_staff_department' created on 'Staff' table.")

# For TRUNCATE and DROP, we need data first. Let's insert some data to demonstrate TRUNCATE later.
# (DML INSERT will be covered soon, but we need it here for TRUNCATE demonstration)
cursor.execute("INSERT INTO Staff (first_name, last_name, department, hire_date, salary) VALUES ('John', 'Doe', 'HR', '2020-01-15', 60000);")
cursor.execute("INSERT INTO Staff (first_name, last_name, department, hire_date, salary) VALUES ('Jane', 'Smith', 'IT', '2021-03-20', 75000);")
conn.commit()

print("Data inserted for TRUNCATE demo.")

# TRUNCATE TABLE (SQLite doesn't have a direct TRUNCATE. We use DELETE FROM for similar effect)
cursor.execute('DELETE FROM Staff WHERE 1=1;') # This effectively truncates in SQLite
print("Table 'Staff' truncated (all rows removed).")

# DROP INDEX
cursor.execute('DROP INDEX idx_staff_department;')
print("Index 'idx_staff_department' dropped.")

# DROP TABLE (demonstrating the command, but we'll recreate for DML examples)
cursor.execute('DROP TABLE Staff;')
print("Table 'Staff' dropped. Will recreate for next section.")

### 2. DML (Data Manipulation Language)

DML commands are used for managing and manipulating data within database objects. They operate on the actual data stored in the tables.

*   **`SELECT`**: Used to retrieve data from one or more tables.
*   **`INSERT`**: Used to add new rows of data into a table.
*   **`UPDATE`**: Used to modify existing data in a table.
*   **`DELETE`**: Used to remove rows of data from a table.
*   **`MERGE`** (or UPSERT): Combines `INSERT` and `UPDATE` operations into a single statement, often used for synchronizing data.
*   **`CALL`**: Used to execute a stored procedure.
*   **`EXPLAIN PLAN`**: Used to show the execution plan of a SQL statement.
*   **`LOCK TABLE`**: Used to lock a table, preventing other users from accessing it during a transaction.

In [ ]:
print("--- DML Commands Examples ---")

# Recreate a table for DML examples
cursor.execute('''
CREATE TABLE Products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    category TEXT,
    price REAL,
    stock_quantity INTEGER
);
''')
print("Table 'Products' recreated.")

# INSERT
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Laptop', 'Electronics', 1200.00, 50);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Mouse', 'Electronics', 25.50, 200);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Keyboard', 'Electronics', 75.00, 150);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Desk Chair', 'Furniture', 150.00, 30);")
conn.commit()
print("Data inserted into 'Products' table.")

# SELECT (basic)
print("\n--- SELECT All Products ---")
cursor.execute('SELECT * FROM Products;')
df_products = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_products)

# SELECT with WHERE clause
print("\n--- SELECT Electronics Products ---")
cursor.execute("SELECT product_name, price FROM Products WHERE category = 'Electronics';")
df_electronics = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_electronics)

# UPDATE
cursor.execute("UPDATE Products SET price = 1250.00 WHERE product_name = 'Laptop';")
conn.commit()
print("\n--- Laptop price updated. SELECT updated row ---")
cursor.execute("SELECT * FROM Products WHERE product_name = 'Laptop';")
df_updated_laptop = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_updated_laptop)

# DELETE
cursor.execute("DELETE FROM Products WHERE product_name = 'Desk Chair';")
conn.commit()
print("\n--- 'Desk Chair' deleted. SELECT all remaining products ---")
cursor.execute('SELECT * FROM Products;')
df_remaining = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_remaining)

# Note: MERGE, CALL, EXPLAIN PLAN, LOCK TABLE are database-specific and not directly supported in all SQLite contexts.
# For example, in databases like PostgreSQL or SQL Server, you might see:
# -- MERGE (SQL Server/Oracle Syntax)
# MERGE INTO TargetTable AS T
# USING SourceTable AS S
# ON T.id = S.id
# WHEN MATCHED THEN UPDATE SET T.col = S.col
# WHEN NOT MATCHED THEN INSERT (cols) VALUES (vals);

# -- EXPLAIN (PostgreSQL/MySQL)
# EXPLAIN SELECT * FROM Products WHERE category = 'Electronics';

### 3. DCL (Data Control Language)

DCL commands are used for managing user permissions and access control within the database. They define who can access what and what operations they can perform.

*   **`GRANT`**: Used to give specific database privileges to a user or role (e.g., `SELECT`, `INSERT`, `UPDATE`, `DELETE` on a table).
*   **`REVOKE`**: Used to remove previously granted database privileges from a user or role.

In [ ]:
print("--- DCL Commands Examples (Conceptual, as SQLite has limited user management) ---")

# GRANT (Conceptual example for a real RDBMS like PostgreSQL, MySQL, Oracle)
# GRANT SELECT, INSERT ON Products TO 'user_analyst';
# GRANT ALL PRIVILEGES ON DATABASE my_database TO 'admin_user';
print("GRANT: Conceptually, it gives privileges like SELECT, INSERT, UPDATE, DELETE to users/roles.")

# REVOKE (Conceptual example)
# REVOKE INSERT ON Products FROM 'user_analyst';
print("REVOKE: Conceptually, it takes away privileges from users/roles.")

print("In SQLite, user management is typically handled at the file system level for standalone databases, or via connection strings for in-memory.")

### 4. TCL (Transaction Control Language)

TCL commands are used to manage transactions, which are sequences of operations performed as a single logical unit of work. Transactions ensure data integrity and consistency.

*   **`COMMIT`**: Used to permanently save the changes made during the current transaction to the database.
*   **`ROLLBACK`**: Used to undo the changes made during the current transaction, restoring the database to its state before the transaction began.
*   **`SAVEPOINT`**: Used to set a point within a transaction to which you can later roll back. This allows partial rollbacks.

In [ ]:
print("--- TCL Commands Examples ---")

# Recreate table and insert some data for TCL demo
cursor.execute('DROP TABLE IF EXISTS Orders;')
cursor.execute('''
CREATE TABLE Orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    total_amount REAL
);
''')
print("Table 'Orders' created for TCL demo.")

# Start a transaction (implied in sqlite3 unless autocommit is off or explicit BEGIN TRANSACTION)
# INSERT data
cursor.execute("INSERT INTO Orders (customer_id, order_date, total_amount) VALUES (101, '2023-10-26', 250.00);")
cursor.execute("INSERT INTO Orders (customer_id, order_date, total_amount) VALUES (102, '2023-10-26', 120.50);")

print("\n--- After two INSERTS (not yet committed) ---")
cursor.execute('SELECT * FROM Orders;')
df_before_commit = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_before_commit)

# COMMIT - Save changes permanently
conn.commit()
print("Changes committed. Data is now permanent.")

# Start another transaction
cursor.execute("INSERT INTO Orders (customer_id, order_date, total_amount) VALUES (103, '2023-10-27', 500.00);")
cursor.execute("INSERT INTO Orders (customer_id, order_date, total_amount) VALUES (104, '2023-10-27', 30.00);")

print("\n--- After two more INSERTS (not yet committed) ---")
cursor.execute('SELECT * FROM Orders;')
df_before_rollback = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_before_rollback)

# ROLLBACK - Undo changes in the current transaction
conn.rollback()
print("Changes rolled back. The last two inserts are undone.")

print("\n--- After ROLLBACK ---")
cursor.execute('SELECT * FROM Orders;')
df_after_rollback = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_after_rollback)

# SAVEPOINT (conceptual, as sqlite3 handles it automatically with nested BEGIN/COMMIT/ROLLBACK)
# In other RDBMS, you might use:
# SAVEPOINT sp1;
# INSERT INTO Orders ...;
# ROLLBACK TO SAVEPOINT sp1;
print("SAVEPOINT: Allows partial rollback within a transaction.")

## SQL Use Cases in Projects with Examples

SQL is fundamental to almost any data-driven project. Here are some key use cases:

### 1. Data Retrieval and Analysis

*   **Description**: Extracting specific data, aggregating it, joining data from multiple tables, and performing complex analytical queries to gain insights.
*   **Project Examples**:
    *   **E-commerce**: Find top-selling products by category, calculate average order value, identify customers with the most purchases.
    *   **Marketing**: Segment customers based on demographics and purchase history, analyze campaign performance.
    *   **Finance**: Generate reports on financial transactions, analyze stock performance, calculate portfolio value.

*   **SQL Example**: Get the total sales for each product category.

In [ ]:
print("--- Use Case: Data Retrieval and Analysis ---")

# Re-insert data into Products for a clear example
cursor.execute('DROP TABLE IF EXISTS Products;')
cursor.execute('''
CREATE TABLE Products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    category TEXT,
    price REAL,
    stock_quantity INTEGER
);
''')
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Laptop', 'Electronics', 1200.00, 50);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Mouse', 'Electronics', 25.50, 200);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Keyboard', 'Electronics', 75.00, 150);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Desk Chair', 'Furniture', 150.00, 30);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Monitor', 'Electronics', 300.00, 80);")
cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES ('Coffee Table', 'Furniture', 80.00, 40);")
conn.commit()

cursor.execute('DROP TABLE IF EXISTS Sales;')
cursor.execute('''
CREATE TABLE Sales (
    sale_id INTEGER PRIMARY KEY,
    product_id INTEGER,
    sale_date DATE,
    quantity_sold INTEGER,
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);
''')
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (1, '2023-10-01', 5);")
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (2, '2023-10-02', 10);")
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (1, '2023-10-03', 3);")
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (4, '2023-10-03', 2);")
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (3, '2023-10-04', 7);")
conn.commit()

print("Tables 'Products' and 'Sales' populated for analysis.")

# SQL Example: Calculate total revenue by category
query = '''
SELECT p.category,
       SUM(p.price * s.quantity_sold) AS total_revenue
FROM Products p
JOIN Sales s ON p.product_id = s.product_id
GROUP BY p.category
ORDER BY total_revenue DESC;
'''
cursor.execute(query)
df_revenue_by_category = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_revenue_by_category)

### 2. Data Management and Manipulation

*   **Description**: Creating, updating, and deleting records; maintaining data integrity; and transforming data for various uses.
*   **Project Examples**:
    *   **CRM Systems**: Adding new customer records, updating customer contact information, deleting inactive accounts.
    *   **Inventory Management**: Updating stock levels, adding new products, removing discontinued items.
    *   **User Authentication**: Storing user credentials, updating passwords.

*   **SQL Example**: Update a product's stock quantity after a sale.

In [ ]:
print("--- Use Case: Data Management and Manipulation ---")

# SQL Example: Update stock quantity for 'Laptop' after selling 1 unit
product_to_update = 'Laptop'
quantity_sold = 1

# Get current stock
cursor.execute("SELECT stock_quantity FROM Products WHERE product_name = ?;", (product_to_update,))
current_stock = cursor.fetchone()[0]
print(f"Current stock for {product_to_update}: {current_stock}")

# Update stock
cursor.execute("UPDATE Products SET stock_quantity = ? WHERE product_name = ?;", (current_stock - quantity_sold, product_to_update))
conn.commit()

# Verify update
cursor.execute("SELECT product_name, stock_quantity FROM Products WHERE product_name = ?;", (product_to_update,))
df_updated_stock = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_updated_stock)

### 3. Database Design and Definition

*   **Description**: Defining the structure of the database, creating tables, defining relationships, constraints, and indexes to ensure efficient storage and retrieval.
*   **Project Examples**:
    *   **Building a new application**: Designing the entire database schema from scratch based on application requirements.
    *   **Database Migration**: Adapting existing schemas for new systems or database platforms.
    *   **Performance Tuning**: Adding indexes to speed up slow queries.

*   **SQL Example**: Creating a new table with appropriate data types and constraints.

In [ ]:
print("--- Use Case: Database Design and Definition ---")

# SQL Example: Create a new 'Customers' table with primary key and unique constraint
cursor.execute('DROP TABLE IF EXISTS Customers;')
cursor.execute('''
CREATE TABLE Customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    registration_date DATE DEFAULT CURRENT_DATE
);
''')
conn.commit()
print("Table 'Customers' created with PRIMARY KEY and UNIQUE constraints.")

# Insert some data to verify
cursor.execute("INSERT INTO Customers (customer_name, email) VALUES ('Alice Johnson', 'alice@example.com');")
cursor.execute("INSERT INTO Customers (customer_name, email) VALUES ('Bob Williams', 'bob@example.com');")
conn.commit()

cursor.execute('SELECT * FROM Customers;')
df_customers = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_customers)

### 4. User and Permission Management

*   **Description**: Granting and revoking access rights to users or roles, ensuring data security and compliance.
*   **Project Examples**:
    *   **Enterprise Applications**: Different user roles (e.g., admin, editor, viewer) have different access levels to various parts of the database.
    *   **Compliance**: Restricting access to sensitive data (e.g., personal identifiable information) to authorized personnel only.

*   **SQL Example**: Granting/Revoking permissions (conceptual in SQLite, but critical in real RDBMS).

In [ ]:
print("--- Use Case: User and Permission Management (Conceptual) ---")

# SQL Example (for a typical RDBMS like PostgreSQL):
# GRANT SELECT ON Products TO sales_team_role;
# REVOKE DELETE ON Customers FROM public;

print("In a multi-user database system (like PostgreSQL, MySQL, SQL Server), DCL commands (`GRANT`, `REVOKE`) are used extensively to manage who can do what. For example, a 'sales_team_role' might get `SELECT` permissions on product and customer data, but only `INSERT` and `UPDATE` on sales orders, without `DELETE` access to prevent accidental data loss.")

### 5. Transaction Management

*   **Description**: Ensuring that a series of database operations are treated as a single, atomic unit. Either all operations succeed (commit) or all are undone (rollback), maintaining database consistency.
*   **Project Examples**:
    *   **Banking Systems**: Transferring money between accounts (debit one, credit another) must be an atomic operation. If one fails, both should be undone.
    *   **Order Processing**: Creating an order, updating inventory, and logging the transaction as a single unit.

*   **SQL Example**: Handling a bank transfer scenario.

In [ ]:
print("--- Use Case: Transaction Management ---")

# Create Accounts table
cursor.execute('DROP TABLE IF EXISTS Accounts;')
cursor.execute('''
CREATE TABLE Accounts (
    account_id INTEGER PRIMARY KEY,
    account_holder TEXT NOT NULL,
    balance REAL NOT NULL
);
''')
cursor.execute("INSERT INTO Accounts (account_holder, balance) VALUES ('Alice', 1000.00);")
cursor.execute("INSERT INTO Accounts (account_holder, balance) VALUES ('Bob', 500.00);")
conn.commit()
print("Table 'Accounts' created and populated.")

print("\n--- Balances before transfer ---")
cursor.execute('SELECT * FROM Accounts;')
df_balances_before = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_balances_before)

# Simulate a transfer: Alice sends Bob 200.00
try:
    # Debit Alice's account
    cursor.execute("UPDATE Accounts SET balance = balance - 200.00 WHERE account_holder = 'Alice';")

    # Imagine an error occurs here, or the system crashes before committing the second step
    # For demonstration, let's simulate a condition that would cause a failure later if we were not careful
    # e.g., if Bob's account didn't exist or a constraint was violated

    # Credit Bob's account
    cursor.execute("UPDATE Accounts SET balance = balance + 200.00 WHERE account_holder = 'Bob';")

    conn.commit() # Commit both operations as a single transaction
    print("Transaction successful: 200.00 transferred from Alice to Bob.")
except Exception as e:
    conn.rollback() # If any error, roll back all changes
    print(f"Transaction failed: {e}. Rolled back changes.")

print("\n--- Balances after transfer (or rollback) ---")
cursor.execute('SELECT * FROM Accounts;')
df_balances_after = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_balances_after)

# Example of a failed transaction (e.g., insufficient funds)
try:
    print("\nAttempting transfer where Alice sends Bob 1500.00 (insufficient funds for Alice)")
    # Debit Alice's account with more than she has
    cursor.execute("UPDATE Accounts SET balance = balance - 1500.00 WHERE account_holder = 'Alice';")

    # In a real scenario, you'd check balance before debiting or use a CHECK constraint
    # For this example, let's just show that if a subsequent action fails, the first is rolled back
    # Let's say, there was an issue crediting Bob's account (e.g., non-existent account)
    # We'll simulate a failure by trying to update a non-existent account or similar problem.
    # In sqlite, a simple way to cause a rollback on error is to explicitly raise an exception
    raise ValueError("Simulated error during credit operation for Bob")

    cursor.execute("UPDATE Accounts SET balance = balance + 1500.00 WHERE account_holder = 'Bob';")
    conn.commit()
    print("Transaction successful (this won't be reached)")
except Exception as e:
    conn.rollback()
    print(f"Transaction failed: {e}. Rolled back changes to ensure consistency.")

print("\n--- Balances after failed transaction attempt ---")
cursor.execute('SELECT * FROM Accounts;')
df_balances_final = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_balances_final)

### 6. Reporting and Business Intelligence

*   **Description**: Generating structured reports, dashboards, and analytical views to support business decision-making.
*   **Project Examples**:
    *   **Sales Dashboards**: Daily, weekly, monthly sales figures, sales by region, product performance.
    *   **Customer Insights**: Reports on customer lifetime value, churn rate, demographics.
    *   **Operational Reports**: Inventory levels, order fulfillment status, employee performance metrics.

*   **SQL Example**: Create a report showing monthly sales trends.

In [ ]:
print("--- Use Case: Reporting and Business Intelligence ---")

# Add more sales data over different months
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (1, '2023-09-10', 2);")
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (3, '2023-09-15', 4);")
cursor.execute("INSERT INTO Sales (product_id, sale_date, quantity_sold) VALUES (2, '2023-11-05', 8);")
conn.commit()

# SQL Example: Monthly sales report
query = '''
SELECT STRFTIME('%Y-%m', sale_date) AS sale_month,
       SUM(s.quantity_sold * p.price) AS monthly_revenue
FROM Sales s
JOIN Products p ON s.product_id = p.product_id
GROUP BY sale_month
ORDER BY sale_month;
'''
cursor.execute(query)
df_monthly_sales = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_monthly_sales)

### 7. Data Integration and ETL (Extract, Transform, Load)

*   **Description**: Moving data between different systems, often involving cleaning, transforming, and loading into a data warehouse or data lake for further analysis.
*   **Project Examples**:
    *   **Data Warehousing**: Extracting data from operational databases, transforming it into a consistent format, and loading it into a data warehouse.
    *   **System Migration**: Moving data from an old system to a new one.
    *   **Data Synchronization**: Keeping data consistent across multiple databases.

*   **SQL Example**: Combining data from a staging table into a main table (conceptual `MERGE` or series of `INSERT`/`UPDATE`).

### 9. SQL CASE Statement

The SQL `CASE` statement is a conditional expression that allows you to apply `IF/THEN/ELSE` logic directly within your SQL queries. It's often used to create new columns based on conditions or to modify output based on specific criteria.

#### Types of CASE Statements:

1.  **Simple `CASE`**: Compares an expression to a set of simple values.
    ```sql
    CASE expression
        WHEN value1 THEN result1
        WHEN value2 THEN result2
        ...
        ELSE default_result
    END
    ```
2.  **Searched `CASE`**: Evaluates a set of boolean expressions.
    ```sql
    CASE
        WHEN condition1 THEN result1
        WHEN condition2 THEN result2
        ...
        ELSE default_result
    END
    ```

`ELSE` is optional. If omitted and no `WHEN` condition is met, `NULL` is returned.

### 8. SQL JOINs

SQL `JOIN` clauses are used to combine rows from two or more tables based on a related column between them. They are essential for retrieving data that is spread across multiple tables in a relational database.

#### Common Types of JOINs:

1.  **`INNER JOIN`**: Returns only the rows that have matching values in both tables.
2.  **`LEFT JOIN` (or `LEFT OUTER JOIN`)**: Returns all rows from the left table, and the matching rows from the right table. If there is no match, `NULL` is returned for the right side columns.
3.  **`RIGHT JOIN` (or `RIGHT OUTER JOIN`)**: Returns all rows from the right table, and the matching rows from the left table. If there is no match, `NULL` is returned for the left side columns. (Note: SQLite does not directly support `RIGHT JOIN` but it can be achieved by reversing the tables in a `LEFT JOIN`).
4.  **`FULL JOIN` (or `FULL OUTER JOIN`)**: Returns all rows when there is a match in one of the tables. Returns all rows from both tables, with `NULL`s for the side that doesn't have a match. (Note: SQLite does not directly support `FULL JOIN` but it can be simulated using `UNION ALL` with `LEFT JOIN` and `RIGHT JOIN`).
5.  **`CROSS JOIN`**: Returns the Cartesian product of the sets of rows from the joined tables. This means it combines every row from the first table with every row from the second table.

In [ ]:
print("--- Use Case: Data Integration and ETL (Conceptual) ---")

# Create a Staging table
cursor.execute('DROP TABLE IF EXISTS Staging_Products;')
cursor.execute('''
CREATE TABLE Staging_Products (
    temp_product_id INTEGER PRIMARY KEY,
    temp_product_name TEXT NOT NULL,
    temp_category TEXT,
    temp_price REAL,
    temp_stock_change INTEGER
);
''')

cursor.execute("INSERT INTO Staging_Products (temp_product_name, temp_category, temp_price, temp_stock_change) VALUES ('Webcam', 'Electronics', 50.00, 100);") # New Product
cursor.execute("INSERT INTO Staging_Products (temp_product_name, temp_category, temp_price, temp_stock_change) VALUES ('Laptop', 'Electronics', 1200.00, 5);") # Stock update
conn.commit()
print("Staging table 'Staging_Products' created and populated.")

# SQL Example (Conceptual MERGE/UPSERT logic using INSERT OR REPLACE or separate UPDATE/INSERT)
print("\n--- Before ETL process ---")
cursor.execute('SELECT product_name, stock_quantity FROM Products;')
df_products_before_etl = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_products_before_etl)

# Simulate ETL process: Add new products and update existing stock
print("\n--- Simulating ETL: Add new products and update stock ---")
for row in cursor.execute('SELECT temp_product_name, temp_category, temp_price, temp_stock_change FROM Staging_Products;'):
    product_name, category, price, stock_change = row

    # Check if product exists
    cursor.execute("SELECT product_id, stock_quantity FROM Products WHERE product_name = ?;", (product_name,))
    existing_product = cursor.fetchone()

    if existing_product:
        # Product exists, update stock
        product_id, current_stock = existing_product
        new_stock = current_stock + stock_change # stock_change could be positive or negative
        cursor.execute("UPDATE Products SET stock_quantity = ? WHERE product_id = ?;", (new_stock, product_id))
        print(f"Updated stock for {product_name} to {new_stock}")
    else:
        # New product, insert it
        cursor.execute("INSERT INTO Products (product_name, category, price, stock_quantity) VALUES (?, ?, ?, ?);", (product_name, category, price, stock_change))
        print(f"Inserted new product: {product_name}")
conn.commit()

print("\n--- After ETL process ---")
cursor.execute('SELECT product_name, stock_quantity FROM Products;')
df_products_after_etl = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df_products_after_etl)

# Close the connection
conn.close()
print("\nDatabase connection closed.")

### 10. SQL Subqueries (Nested Queries)

A subquery (or inner query) is a query nested inside another SQL query. It can be used to return data that will be used by the main query as a condition to further restrict the data to be retrieved.

Subqueries can be used with `SELECT`, `INSERT`, `UPDATE`, and `DELETE` statements. They can also be used with operators like `=`, `<`, `>`, `>=`, `<=`, `IN`, `NOT IN`, `EXISTS`, `NOT EXISTS`, etc.

#### Common Uses of Subqueries:

1.  **In the `WHERE` clause**: To filter results based on a condition derived from another query.
2.  **In the `FROM` clause (Derived Table)**: To treat the result of a subquery as a temporary table.
3.  **In the `SELECT` clause (Scalar Subquery)**: To return a single value for each row in the outer query.